In [9]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

# === 1️⃣ Koneksi ke MongoDB ===
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

# === 2️⃣ Hapus semua data lama agar tidak duplikat ===
courses_collection.delete_many({})

# === 3️⃣ Gunakan bulk_write untuk operasi masal (Insert, Update, Delete) ===
operations = [
    # ---- INSERT: Tambah beberapa data kursus ----
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}),

    # ---- UPDATE: Ubah beberapa data sekaligus ----
    UpdateOne({'course': 'CS 102'}, {'$set': {'enrollments': 28}}),       # ubah enrollments CS 102 jadi 28
    UpdateOne({'course': 'Math 101'}, {'$inc': {'enrollments': 5}}),     # tambah enrollments Math 101 sebanyak 5

    # ---- DELETE: Hapus 1 data ----
    DeleteOne({'course': 'Physics 202'})
]

# Jalankan operasi sekaligus
result = courses_collection.bulk_write(operations)
print("✅ Bulk operations completed successfully.\n")

# === 4️⃣ Tampilkan hasil akhir ===
print("=== Data Kursus Saat Ini ===")
for course in courses_collection.find():
    print(course)

✅ Bulk operations completed successfully.

=== Data Kursus Saat Ini ===
{'_id': ObjectId('690c16ef979cdf85d87430ad'), 'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics'}
{'_id': ObjectId('690c16ef979cdf85d87430ae'), 'course': 'CS 102', 'enrollments': 28, 'department': 'Computer Science'}
{'_id': ObjectId('690c16ef979cdf85d87430af'), 'course': 'History 201', 'enrollments': 20, 'department': 'History'}


In [3]:
# Query for courses with enrollments greater than 20
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(course)

{'_id': ObjectId('690c12bf979cdf85d87430a8'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690c12bf979cdf85d87430a9'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [4]:
# Query courses in Computer Science or Mathematics departments
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(course)

{'_id': ObjectId('690c12bf979cdf85d87430a8'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690c12bf979cdf85d87430a9'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [5]:
# Average enrollment per department using aggregation
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Physics', 'average_enrollment': 15.0}
{'_id': 'Computer Science', 'average_enrollment': 25.0}
{'_id': 'History', 'average_enrollment': 20.0}
{'_id': 'Mathematics', 'average_enrollment': 30.0}


In [6]:
# Maximum enrollment per department
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)


{'_id': 'Computer Science', 'max_enrollment': 25}
{'_id': 'History', 'max_enrollment': 20}
{'_id': 'Mathematics', 'max_enrollment': 30}
{'_id': 'Physics', 'max_enrollment': 15}


In [7]:
# Projection to rename fields
pipeline = [
    {'$project': {'course_name': '$course', 'department_name': '$department', 'enrollments': 1}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('690c12bf979cdf85d87430a8'), 'enrollments': 30, 'course_name': 'Math 101', 'department_name': 'Mathematics'}
{'_id': ObjectId('690c12bf979cdf85d87430a9'), 'enrollments': 25, 'course_name': 'CS 102', 'department_name': 'Computer Science'}
{'_id': ObjectId('690c12bf979cdf85d87430aa'), 'enrollments': 20, 'course_name': 'History 201', 'department_name': 'History'}
{'_id': ObjectId('690c12bf979cdf85d87430ab'), 'enrollments': 15, 'course_name': 'Physics 202', 'department_name': 'Physics'}


In [8]:
# Adding enrollment category field based on enrollments
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'}}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('690c12bf979cdf85d87430a8'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'_id': ObjectId('690c12bf979cdf85d87430a9'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'_id': ObjectId('690c12bf979cdf85d87430aa'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrollment_category': 'low'}
{'_id': ObjectId('690c12bf979cdf85d87430ab'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'enrollment_category': 'low'}


In [10]:
pipeline = [
    {'$group': {'_id': '$department', 'total_courses': {'$sum': 1}}}
]

for result in courses_collection.aggregate(pipeline):
    print(result)


{'_id': 'Computer Science', 'total_courses': 1}
{'_id': 'History', 'total_courses': 1}
{'_id': 'Mathematics', 'total_courses': 1}


In [11]:
pipeline = [
    {'$match': {'department': 'Computer Science', 'enrollments': {'$gt': 25}}},
    {'$group': {'_id': '$department', 'total_high_enrollment_courses': {'$sum': 1}}}
]

for result in courses_collection.aggregate(pipeline):
    print(result)


{'_id': 'Computer Science', 'total_high_enrollment_courses': 1}


In [12]:
# Misalkan sudah ada koleksi students
students_collection = db['students']

# Contoh data students (jika belum ada)
students_collection.insert_many([
    {'student_id': 1, 'name': 'Alice', 'enrolled_course': 'Math 101'},
    {'student_id': 2, 'name': 'Bob', 'enrolled_course': 'CS 102'},
    {'student_id': 3, 'name': 'Charlie', 'enrolled_course': 'CS 102'},
    {'student_id': 4, 'name': 'Diana', 'enrolled_course': 'History 201'}
])

# Menggabungkan (join) koleksi courses dan students
pipeline = [
    {'$lookup': {
        'from': 'students',
        'localField': 'course',
        'foreignField': 'enrolled_course',
        'as': 'student_details'
    }}
]

for result in courses_collection.aggregate(pipeline):
    print(result)


{'_id': ObjectId('690c16ef979cdf85d87430ad'), 'course': 'Math 101', 'enrollments': 35, 'department': 'Mathematics', 'student_details': [{'_id': ObjectId('690c1d8c979cdf85d87430b1'), 'student_id': 1, 'name': 'Alice', 'enrolled_course': 'Math 101'}]}
{'_id': ObjectId('690c16ef979cdf85d87430ae'), 'course': 'CS 102', 'enrollments': 28, 'department': 'Computer Science', 'student_details': [{'_id': ObjectId('690c1d8c979cdf85d87430b2'), 'student_id': 2, 'name': 'Bob', 'enrolled_course': 'CS 102'}, {'_id': ObjectId('690c1d8c979cdf85d87430b3'), 'student_id': 3, 'name': 'Charlie', 'enrolled_course': 'CS 102'}]}
{'_id': ObjectId('690c16ef979cdf85d87430af'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'student_details': [{'_id': ObjectId('690c1d8c979cdf85d87430b4'), 'student_id': 4, 'name': 'Diana', 'enrolled_course': 'History 201'}]}
